# Item Types Guide: Variable, Number, Category, and Filter

This notebook explains when to use each item type in pollywog. You'll learn:
- The four core item types and their purposes
- When each type is visible in Leapfrog
- Practical examples of each type
- Decision flow for choosing the right type

In [ ]:
import pollywog as pw
from pollywog.core import CalcSet, Variable, Number, Category, Filter, If
from pollywog.display import display_calcset, set_theme

set_theme("light")

## Overview: The Four Item Types

| Type | Output | Visible Outside Calculator? | Use Case |
|------|--------|----------------------------|----------|
| **Variable** | Numeric or Text | ❌ No | Intermediate calculations |
| **Number** | Numeric | ✅ Yes | Final numeric outputs |
| **Category** | Text/Label | ✅ Yes | Final categorical outputs |
| **Filter** | Boolean | ✅ Yes (as filter) | Data inclusion rules |

**Key distinction:** 
- **Variable**: Only visible in calculator UI, cannot visualize in 3D or export
- **Number/Category/Filter**: Available everywhere in Leapfrog

## 1. Variable: Intermediate Calculations

Use `Variable` for intermediate steps you need for calculations but won't visualize or export.

In [ ]:
# Example: Multi-step calculation with intermediate variables
workflow_with_vars = CalcSet([
    # Step 1: Clean data (intermediate - don't need to export)
    Variable("Au_clean", "clamp([Au], 0)", comment_equation="Remove negatives"),
    Variable("Au_capped", "clamp([Au_clean], 0, 100)", comment_equation="Cap at 100 g/t"),
    
    # Step 2: Transform (intermediate)
    Variable("Au_log", "log([Au_capped] + 0.01)", comment_equation="Log transform"),
    
    # Step 3: Final output (this one we want to visualize)
    Number("Au_prepared", "[Au_log] * 1.1", comment_equation="Prepared for kriging"),
])

display_calcset(workflow_with_vars)

print("\nNote: Au_clean, Au_capped, and Au_log are Variables.")
print("They're visible in the calculator but won't clutter your block model variable list.")
print("Only Au_prepared (a Number) is available for 3D visualization and export.")

## 2. Number: Final Numeric Outputs

Use `Number` for numeric results you want to visualize, export, or use in other Leapfrog tools.

In [ ]:
# Example: Resource estimation outputs
final_outputs = CalcSet([
    # These are all final outputs we want to visualize
    Number("Au_est", "exp([Au_log_kriged]) - 0.01", comment_equation="Back-transformed estimate"),
    Number("Au_diluted", "[Au_est] * 0.95", comment_equation="5% dilution"),
    Number("Au_recovered", "[Au_diluted] * [recovery]", comment_equation="Metallurgical recovery"),
    Number("tonnage", "[volume] * [density]", comment_equation="Block tonnage"),
    Number("metal_tonnes", "[tonnage] * [Au_recovered] / 1000", comment_equation="Contained Au (tonnes)"),
    Number("revenue", "[metal_tonnes] * [Au_price] * 31.1035", comment_equation="Estimated revenue (USD)"),
])

display_calcset(final_outputs)

print("\nAll of these Number items will be:")
print("✅ Available for 3D visualization")
print("✅ Exportable to CSV/other formats")
print("✅ Usable in other Leapfrog evaluations")
print("✅ Visible in the block model variable list")

## 3. Category: Text/Categorical Outputs

Use `Category` for text labels and classifications you want to visualize or export.

In [ ]:
# Example: Classifications for visualization
classifications = CalcSet([
    # Simple classification
    Category(
        "ore_type",
        If(("[Au] > 0.5", "'ore'"), "'waste'"),
        comment_equation="Binary ore/waste classification"
    ),
    
    # Multi-class
    Category(
        "ore_grade",
        If(
            [
                ("[Au] < 0.3", "'waste'"),
                ("[Au] < 1.0", "'low'"),
                ("[Au] < 3.0", "'medium'"),
            ],
            "'high'"
        ),
        comment_equation="4-class grade classification"
    ),
    
    # Domain assignment
    Category(
        "processing_domain",
        If(
            [
                ("[rock_type] == 'oxide' and [Au] > 0.3", "'oxide_mill'"),
                ("[rock_type] == 'oxide'", "'heap_leach'"),
                ("[rock_type] == 'sulfide' and [Au] > 0.5", "'sulfide_mill'"),
            ],
            "'waste'"
        ),
        comment_equation="Processing domain assignment"
    ),
])

display_calcset(classifications)

print("\nCategory items are perfect for:")
print("✅ Color-coded 3D visualization (each category gets a color)")
print("✅ Filtering and selection in Leapfrog")
print("✅ Reporting and statistics by category")
print("✅ Mine planning and scheduling")

## 4. Filter: Boolean Inclusion Rules

Use `Filter` for true/false conditions that determine which data to include in calculations.

In [ ]:
# Example: Filters for data quality and inclusion
filters = CalcSet([
    # Data quality filters
    Filter("has_valid_Au", "is_normal([Au])", comment_equation="Au data exists and is valid"),
    Filter("has_valid_density", "is_normal([density])", comment_equation="Density data is valid"),
    Filter("complete_data", "[has_valid_Au] and [has_valid_density]", comment_equation="All required data present"),
    
    # Threshold filters
    Filter("above_cutoff", "[Au] > 0.5", comment_equation="Economically viable"),
    Filter("within_pit", "[elevation] > [pit_floor_elevation]", comment_equation="Inside pit shell"),
    
    # Combined filter for estimation
    Filter(
        "use_in_estimate",
        "[complete_data] and [above_cutoff] and [within_pit]",
        comment_equation="Include in resource estimation"
    ),
])

display_calcset(filters)

print("\nFilters are used for:")
print("✅ Excluding invalid/missing data from calculations")
print("✅ Applying cutoff grades")
print("✅ Defining estimation domains")
print("⚠️  Less common in block models (usually classify everything)")
print("✅ More useful for drillhole data processing")

## 5. When to Use Each Type: Decision Flow

```
Is the output true/false?
└─ Yes → Use Filter
└─ No ↓

Is the output text/label?
└─ Yes ↓
    Will you visualize or export it?
    └─ Yes → Use Category
    └─ No → Use Variable
└─ No ↓

Is the output numeric?
└─ Yes ↓
    Will you visualize or export it?
    └─ Yes → Use Number
    └─ No → Use Variable (intermediate calculation)
```

## 6. Complete Example: All Types Working Together

A realistic workflow using all four types appropriately.

In [ ]:
# Complete resource estimation workflow
complete_workflow = CalcSet([
    # Filters: Define what data to use
    Filter("valid_Au", "is_normal([Au]) and [Au] >= 0"),
    Filter("valid_density", "is_normal([density]) and [density] > 1.5 and [density] < 4.0"),
    
    # Variables: Intermediate calculations (won't export these)
    Variable("Au_capped", "clamp([Au], 0, 100)", comment_equation="Cap outliers"),
    Variable("Au_kriged_log", "[Au_log_estimate]", comment_equation="From kriging"),
    
    # Numbers: Final numeric outputs (will visualize these)
    Number("Au_est", "exp([Au_kriged_log]) - 0.01", comment_equation="Back-transformed grade"),
    Number("Au_diluted", "[Au_est] * 0.95", comment_equation="5% dilution"),
    Number("tonnage", "[block_volume] * [density]", comment_equation="Block tonnage"),
    Number("Au_metal", "[tonnage] * [Au_diluted] / 31.1035", comment_equation="Troy ounces"),
    
    # Categories: Classifications (for visualization and reporting)
    Category(
        "resource_class",
        If(
            [
                ("[Au_diluted] < 0.3", "'waste'"),
                ("[Au_diluted] < 1.0", "'low_grade'"),
                ("[Au_diluted] < 3.0", "'medium_grade'"),
            ],
            "'high_grade'"
        ),
        comment_equation="Resource classification"
    ),
    Category(
        "data_quality",
        If(
            [
                ("not [valid_Au]", "'invalid_Au'"),
                ("not [valid_density]", "'invalid_density'"),
            ],
            "'good'"
        ),
        comment_equation="QA/QC flag"
    ),
])

display_calcset(complete_workflow)

print("\n=== Workflow Summary ===")
print(f"Filters: {len([i for i in complete_workflow.items if i.item_type == 'filter'])} (data quality checks)")
print(f"Variables: {len([i for i in complete_workflow.items if i.item_type == 'variable'])} (intermediate steps)")
print(f"Numbers: {len([i for i in complete_workflow.items if i.item_type == 'calculation' and i.calculation_type == 'number'])} (final numeric outputs)")
print(f"Categories: {len([i for i in complete_workflow.items if i.item_type == 'calculation' and i.calculation_type == 'category'])} (classifications)")

complete_workflow.to_lfcalc("complete_workflow.lfcalc")

## Summary: Quick Reference

**Variable**
- ❌ Not visible outside calculator
- ✅ Use for intermediate calculations
- ✅ Keeps interface clean
- Example: `Variable("Au_log", "log([Au] + 0.01)")`

**Number**
- ✅ Visible everywhere in Leapfrog
- ✅ Use for final numeric outputs
- ✅ Can visualize, export, use in other tools
- Example: `Number("Au_est", "exp([Au_log_kriged])")`

**Category**
- ✅ Visible everywhere in Leapfrog
- ✅ Use for text labels and classifications
- ✅ Great for color-coded visualization
- Example: `Category("ore_type", If(("[Au] > 0.5", "'ore'"), "'waste'"))`

**Filter**
- ✅ Visible everywhere in Leapfrog
- ✅ Use for boolean inclusion/exclusion
- ✅ More common in drillhole processing
- Example: `Filter("is_ore", "[Au] > 0.5")`

**Rule of thumb:**
- If you'll visualize it → Number or Category
- If it's just a step → Variable
- If it's true/false → Filter